# Hugging Face example

In this notebook, we will explore how to load a Hugging Face model, specifically the SmolLM3-3 using the configuration settings defined in our project.


## Setup


### Load Local Modules


In [1]:
from pathlib import Path
import sys
import subprocess


def get_project_root() -> Path:
    return Path(
        subprocess.check_output(
            ["git", "rev-parse", "--show-toplevel"],
            cwd=".",
            stderr=subprocess.STDOUT,
        )
        .strip()
        .decode()
    )


project_root = get_project_root()
src_dir = project_root / "src"

if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

try:
    from utils.seed_utils import set_seed
except ImportError:
    raise ImportError("Cannot import module. Make sure that the project is on the path")

SEED = 42
set_seed(SEED)

### Setup logs and config

Here's the configuration used:

```yaml
logs:
  _target_: schemas.log_schema.LogSchema
  sink: ${path_config:run_log}
  level: INFO
  format: "{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {thread.name} | {name}:{function}:{line} - {message}"
  rotation: 10 MB
  retention: 7 days
  colorize: true

model:
  _target_: models.smol_lm3.smol_lm3_model.SmolLM3Model
```


In [2]:
from configs.app_config import AppConfig
from utils.log_utils import setup_logs


def setup(
    config_dir: str | Path | None = None,
    config_file: str = "settings.yaml",
    overrides: list[str] | None = None,
):
    AppConfig.load(config_dir=config_dir, config_file=config_file, overrides=overrides)
    setup_logs()


setup()

2025-10-07 11:12:34.327 | SUCCESS  | utils.file_utils:create_dir:125 - Created directory: /home/jmaubras/OneDrive/Bureau/Pro/Aluci-Perso/aluci-ai-template/logs/20251007/111233
2025-10-07 11:12:35.429 | SUCCESS  | utils.file_utils:create_dir:125 - Created directory: /home/jmaubras/OneDrive/Bureau/Pro/Aluci-Perso/aluci-ai-template/logs/20251007/111233/configs
/home/jmaubras/miniconda3/envs/pyenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-07 11:12:37.389 | INFO     | utils.hf_utils:get_huggingface_model:41 - Loading tokenizer for 'HuggingFaceTB/SmolLM3-3B'...
2025-10-07 11:12:38.043 | INFO     | utils.hf_utils:get_huggingface_model:44 - Loading model 'HuggingFaceTB/SmolLM3-3B'...
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.97it/s]
2025-10-07 11:12:39.765 | SUCCESS  | utils

## Load SmolLM3-3B Model


In [3]:
from utils.log_utils import log

log(message=AppConfig.settings["model"].model)

2025-10-07 11:12:39.776 | DEBUG    | __main__:<module>:3 - SmolLM3ForCausalLM(
  (model): SmolLM3Model(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-35): 36 x SmolLM3DecoderLayer(
        (self_attn): SmolLM3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): SmolLM3MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): SmolLM3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): SmolL

## Playground

In [4]:
tokenizer = AppConfig.settings["model"].tok_or_proc
model = AppConfig.settings["model"].model

messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
log(message=tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1] :]))

2025-10-07 11:12:41.067 | DEBUG    | __main__:<module>:16 - <think>
Okay, the user is asking, "Who are you?" I need to respond in a way that's helpful and informative. Let me start by introducing myself as an AI assistant. I should


## Here, the rest of your code. Enjoy !
